---
### 도서 : [tensorflow 2 로 자연처리]

#
Tensorflow 
drop out
Dense

In [1]:
## Forrest Park's notion  ## Updated 2024.07.03
### Service  Text Coloring option
from Functions import Service as S
def blue(str):return S.colored_text(str,'blue')
def yellow(str):return S.colored_text(str,'yellow')
def red(str):return S.colored_text(str,'red')
def green(str):return S.colored_text(str,'green')
def imd(str):return S.imd(green(str))
## 자연어처리 패키지 설치 
def NLPInstalls():
    import subprocess,sys
    import warnings ; warnings.filterwarnings('ignore')
    # pip가 없으면 pip를 설치
    try:import pip
    except ImportError:
        print("Install pip for python3")
        subprocess.call(['sudo', 'apt-get', 'install', 'python3-pip'])
    
    # tweepy 없으면 tweepy 설치
    try:import tweepy        
    except ModuleNotFoundError:
        print("Install tweepy")
        subprocess.call([sys.executable, "-m", "pip", "install", 'tweepy==3.10.0'])
    finally:import tweepy 
    
    # konlpy 없으면 konlpy 설치
    try:import konlpy
    except ModuleNotFoundError: 
        print("Install konlpy")
        subprocess.call([sys.executable, "-m", "pip", "install", 'konlpy'])
    finally:import konlpy
    
    # eunjeon 없으면 eunjeon 설치
    try:import eunjeon
    except ModuleNotFoundError: 
        print("Install eunjeon : eunjeon")
        subprocess.call([sys.executable, "-m", "pip", "install", 'eunjeon'])
    finally:import konlpy
    
    # datasets 없으면 datasets를 설치
    try:import datasets
    except ModuleNotFoundError: 
        print("Install datasets : datasets")
        subprocess.call([sys.executable, "-m", "pip", "install", 'datasets'])
    finally:import datasets
    
    # pytorch 없으면 pytorch 설치
    try:import torch
    except ModuleNotFoundError: 
        print("Install torch : pytorch")
        subprocess.call([sys.executable, "-m", "pip", "install", 'pytorch'])
    finally:import torch
    
    # transformers 없으면 transformers 설치
    try:import transformers
    except ModuleNotFoundError: 
        print("Install transformer : transformers")
        subprocess.call([sys.executable, "-m", "pip", "install", 'transformers'])
    finally:import transformers
        
    # UMAP 없으면 UMAP 설치
    try:import umap
    except ModuleNotFoundError: 
        print("Install umap : umap")
        subprocess.call([sys.executable, "-m", "pip", "install", 'umap'])
    finally:import umap
        
    # UMAP 없으면 UMAP 설치
    try:from umap import UMAP
    except ImportError: 
        print("Install umap : umap-learn")
        subprocess.call([sys.executable, "-m", "pip", "install", 'umap-learn'])
    finally:import umap
   
# NLPInstalls()
print(yellow(f"📌 - PROGRAM START \n\t"))
## GPU setting (in MacOS)
import torch
print(yellow(f"📌 - MPS 장치를 지원 Build 여부 : "),torch.backends.mps.is_built())
print(yellow(f"📌 - MPS 장치 사용가능 여부 : "),torch.backends.mps.is_available())
print(yellow(f"📌 - GPU 사용Start"))
# device = torch.device("mps")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#### Jupyter Basic Setting ####
import pandas as pd,numpy as np
import matplotlib.pyplot as plt,seaborn as sns  # 시각화
import warnings; warnings.filterwarnings('ignore')  # 경고 무시
import sys,os 
from pprint import pprint


📌 - PROGRAM START 
	
📌 - MPS 장치를 지원 Build 여부 :  True
📌 - MPS 장치 사용가능 여부 :  True
📌 - GPU 사용Start


---
### Tensor flow 간단한 자연어 처리 예시 01 page 53

In [2]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import preprocessing 


samples = [' 너 오늘 이뻐 보인다',
           '나는 오늘 기분이 더러워 ',
           '끝내주는데, 좋은 일이 있나봐',
           '나 좋은 일이 생겼어',
           '아 오늘 진짜 짜증나',
           '환상적인데, 정말 좋은 거 같아 ',
           ]
## labeling 
labels = [[1],[0],[1],[1],[0],[1]]
## tokenize
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)

sequences = tokenizer.texts_to_sequences(samples)
print(yellow(f"📌 - text =>sequences"))
pprint([(f"{i} => {j}") for i,j in zip(samples,sequences)])
word_index = tokenizer.word_index
print(yellow(f"\n📌 - word index"))
pprint(word_index)


📌 - text =>sequences
[' 너 오늘 이뻐 보인다 => [4, 1, 5, 6]',
 '나는 오늘 기분이 더러워  => [7, 1, 8, 9]',
 '끝내주는데, 좋은 일이 있나봐 => [10, 2, 3, 11]',
 '나 좋은 일이 생겼어 => [12, 2, 3, 13]',
 '아 오늘 진짜 짜증나 => [14, 1, 15, 16]',
 '환상적인데, 정말 좋은 거 같아  => [17, 18, 2, 19, 20]']

📌 - word index
{'같아': 20,
 '거': 19,
 '기분이': 8,
 '끝내주는데': 10,
 '나': 12,
 '나는': 7,
 '너': 4,
 '더러워': 9,
 '보인다': 6,
 '생겼어': 13,
 '아': 14,
 '오늘': 1,
 '이뻐': 5,
 '일이': 3,
 '있나봐': 11,
 '정말': 18,
 '좋은': 2,
 '진짜': 15,
 '짜증나': 16,
 '환상적인데': 17}


---
### 추가로 모델 구축 및 모델 학습에 피요한 변수 정의 

In [3]:
batch_size = 2
num_epochs = 6
vocab_size =len(word_index) +1
emb_size = 128
hidden_dimension = 256
output_dimension = 1


---
### 데이터를 통과 시킬 모델 구현
- 입력값을 임베딩하는 embedding 층 
- 임베딩된 각 단어의 벡터를 평균하기 위해 람다 층을 사용



In [4]:
model = tf.keras.Sequential()
model.add(layers.Embedding(vocab_size,emb_size,input_length= 4))
model.add(layers.Lambda(lambda x: tf.reduce_mean(x,axis= 1)))
model.add(layers.Dense(hidden_dimension, activation='relu'))
model.add(layers.Dense(output_dimension,activation ="sigmoid"))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

---
### 학습 모델 compile 
- optimizer : Adam 
- loss : 이진 교차 엔트로피 손실 함수 
- 모델 성능 측정 : accuracy

In [5]:
model.compile(optimizer =tf.keras.optimizers.Adam(0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

---
### 모델 학습 하기 


In [6]:
# from keras.preprocessing.sequence import pad_sequences

# #시퀀스 패딩
# padded_sequences = pad_sequences(sequences, padding='post')
# sequences = np.array(padded_sequences)
# labels = np.array(labels)
# model.fit(sequences, labels, epochs=num_epochs, batch_size=batch_size)

---
###
변환모듈 
- 파이프라인
- 특징 추출
- 데이터 전처리
- 차원축소 



In [20]:
import sklearn
sklearn.__version__
from sklearn.datasets import load_iris
iris_dataset = load_iris()
print("iris_dataset : {}".format(iris_dataset.keys()))
print("shape of data: {}".format(iris_dataset['data'].shape))
# print(iris_dataset['data'])
print(iris_dataset['feature_names'])

print(iris_dataset['target'])
print(iris_dataset['target_names'])
print(iris_dataset['DESCR'])


iris_dataset : dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])
shape of data: (150, 4)
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
['setosa' 'versicolor' 'virginica']
.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
             

---
### 사이킷 런을 이용한 데이터 분리



In [27]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_label, test_label = \
    train_test_split(
        iris_dataset['data'],
        iris_dataset['target'],
        test_size=0.25,
        random_state=42
        )
print(f"shaep of train_input : {train_input.shape}")
print(f"shaep of test_input : {test_input.shape}")
print(f"shaep of train_lb : {train_label.shape}")
print(f"shaep of test_lb : {test_label.shape}")



shaep of train_input : (112, 4)
shaep of test_input : (38, 4)
shaep of train_lb : (112,)
shaep of test_lb : (38,)


---
### page 58 비지도학습 을 이용한 text 처리 기법 
- CountVectorize:
    * 각 텍스트에서 횟수를 기준으로 특징을 추출하는 방법 
- TfidfVectorizer: 
    * TF-IDF 라는 값을 사용해 텍스트에서 특징을 추출 하는 방법 

- HashingVectorizer:
    * Count 와 같은 방식이나 텍스트를 처리할때 해시 함수를 사용해 시간을 줄임


In [76]:
## 
from sklearn.feature_extraction.text import CountVectorizer
text_data = ['나는 배가 고프다','내일 점심 뭐먹지','내일 공부해야겠다','점심먹고 공부해야지']
count_vectorizer =CountVectorizer()
count_vectorizer.fit(text_data)
print(count_vectorizer.vocabulary_)
sentence = [text_data[0]] ## 
print(sentence)
print(count_vectorizer.transform(sentence).toarray())


def countVectorize(text_data):
    from sklearn.feature_extraction.text import CountVectorizer
    count_vectorizer =CountVectorizer()
    count_vectorizer.fit([text_data])
    print(yellow(f"vocab : \n"), red(count_vectorizer.vocabulary_))
    print()
    sentence =[[text_data][0]] ## 
    print(sentence)
    print(yellow(f"Vectorized Sentence :\n"),count_vectorizer.transform(sentence).toarray())
    

{'나는': 3, '배가': 6, '고프다': 0, '내일': 4, '점심': 7, '뭐먹지': 5, '공부해야겠다': 1, '점심먹고': 8, '공부해야지': 2}
['나는 배가 고프다']
[[1 0 0 1 0 0 1 0 0]]


---
### 신문기사 데이터에 적용해보자 . 


In [2]:
import pandas as pd
news = pd.read_csv("./NewsData/경제_20240704_10시17분.csv")
# print(news.columns)
test_article  = news.iloc[0]['content']
# print(test_article)
from Functions import NLP_Service as N
N.countVectorize(test_article)

CountVectorize 함수로로 토큰화합니다. 
vocab : 
 {'정책자금': 463, '상환연장': 281, '금융지원': 116, '3종': 18, '세트': 301, '추진5대': 519, '고정비용': 91, '지원': 489, '경영부담': 74, '매출': 205, '기반': 122, '확충': 583, '규제합리화': 111, '방안': 226, '추진': 518, '사회안전망': 272, '가입독려': 44, '소상공인': 308, '실효성': 330, '의문': 385, '장기적': 430, '대책': 166, '필요': 548, '서울': 291, '뉴시스': 156, '추상철': 517, '기자': 126, '3일': 17, '오후': 361, '서울의': 292, '전통시장에서': 446, '상인이': 278, '손님을': 315, '기다리고': 121, '있다': 421, '2024': 11, '06': 3, '03': 1, 'scchoo': 41, 'newsis': 38, 'com': 32, '세종': 299, '김동현': 134, '용윤신': 372, '임하은': 418, '정부가': 456, '올해': 362, '하반기부터': 552, '소상공인의': 313, '경영': 73, '애로를': 338, '해소하기': 567, '위해': 377, '금융지원과': 117, '배달료': 231, '임대료': 412, '전기료': 443, '5대': 26, '고정비용에': 92, '대한': 176, '부담을': 246, '낮추는': 140, '방안을': 228, '본격화한다': 242, '이번': 390, '대책은': 168, '코로나19': 525, '사태': 271, '이후': 403, '고금리': 85, '장기화로': 432, '채무가': 503, '누적되고': 155, '준비': 478, '부족으로': 249, '창업과': 502, '폐업을': 539, '반복하며': 218, '고통을': 93, '받고': 220, '있는': 4

---
### TfidfVectorizer
- TF ( Term Frequency )특정단어 하나가 데이터 안에서 등장하는 횟수
- DF (Document Frequency): 문서 빈도값. 특ㅈ3ㅓㅇ단어가 여러데이터에 자주 등장하는지를 알려주는 지표
- IDF (Inverse Document Frequency ) DF 의 역수 : 다른데이터에 등장하지않을수록 커짐. 
- TF-IDF : TF 와 IDF 를 곱한값. 해당문서에 자주 등장하집만 다른 문서에는 많이 없는 단어일수록 높은 값. 
- 조사, 지시대명사는 TF는 크지만 IDF 값은 작아지므로 counterVectorize 가 가진 문제점 해결


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer


text_data = ['나는 배가 고프다','내일 점심 뭐먹지','내일 공부해야겠다','점심먹고 공부해야지']
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(text_data)
print(tfidf_vectorizer.vocabulary_)
sentence = [text_data[0]] ## 
print(sentence)
print(tfidf_vectorizer.transform(sentence).toarray())



def tfidfVectorize(text_data):
    from sklearn.feature_extraction.text import TfidfVectorizer
    tfidf_vectorizer =TfidfVectorizer()
    tfidf_vectorizer.fit([text_data])
    print(yellow(f"vocab : \n"), red(tfidf_vectorizer.vocabulary_))

    sentence =[[text_data][0]] ## 
    print(sentence)
    print(yellow(f"Vectorized Sentence :\n"),tfidf_vectorizer.transform(sentence).toarray())


import pandas as pd
news = pd.read_csv("./NewsData/경제_20240704_10시17분.csv")
# print(news.columns)
test_article  = news.iloc[0]['content']
# print(test_article)
from Functions import NLP_Service as N
N.tfidfVectorize(test_article)


{'나는': 3, '배가': 6, '고프다': 0, '내일': 4, '점심': 7, '뭐먹지': 5, '공부해야겠다': 1, '점심먹고': 8, '공부해야지': 2}
['나는 배가 고프다']
[[0.57735027 0.         0.         0.57735027 0.         0.
  0.57735027 0.         0.        ]]
TF-IDF 함수로로 토큰화합니다. 
vocab : 
 {'정책자금': 463, '상환연장': 281, '금융지원': 116, '3종': 18, '세트': 301, '추진5대': 519, '고정비용': 91, '지원': 489, '경영부담': 74, '매출': 205, '기반': 122, '확충': 583, '규제합리화': 111, '방안': 226, '추진': 518, '사회안전망': 272, '가입독려': 44, '소상공인': 308, '실효성': 330, '의문': 385, '장기적': 430, '대책': 166, '필요': 548, '서울': 291, '뉴시스': 156, '추상철': 517, '기자': 126, '3일': 17, '오후': 361, '서울의': 292, '전통시장에서': 446, '상인이': 278, '손님을': 315, '기다리고': 121, '있다': 421, '2024': 11, '06': 3, '03': 1, 'scchoo': 41, 'newsis': 38, 'com': 32, '세종': 299, '김동현': 134, '용윤신': 372, '임하은': 418, '정부가': 456, '올해': 362, '하반기부터': 552, '소상공인의': 313, '경영': 73, '애로를': 338, '해소하기': 567, '위해': 377, '금융지원과': 117, '배달료': 231, '임대료': 412, '전기료': 443, '5대': 26, '고정비용에': 92, '대한': 176, '부담을': 246, '낮추는': 140, '방안을': 228, '본격화한다': 242, '이

---
### 자연어 Tokenizing 도구 들 
- 영어 토크나이징 라이브러리 :
    *NLTK 

In [3]:
import nltk
nltk.download() ## all 말뭉치 50개 다운로드 받음. 

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [6]:
## 단어 단위 토크나이징
from nltk.tokenize import word_tokenize
sentence = '''natural language processing(nlp) is a subfield of computer science,
imformation engineering, and artificial intelligence concerned with the interactions
between computers and human  (natural) languages, in particular how to program computers
to process and analyze large amounts of natural language data 
'''
print(word_tokenize(sentence))


['natural', 'language', 'processing', '(', 'nlp', ')', 'is', 'a', 'subfield', 'of', 'computer', 'science', ',', 'imformation', 'engineering', ',', 'and', 'artificial', 'intelligence', 'concerned', 'with', 'the', 'interactions', 'between', 'computers', 'and', 'human', '(', 'natural', ')', 'languages', ',', 'in', 'particular', 'how', 'to', 'program', 'computers', 'to', 'process', 'and', 'analyze', 'large', 'amounts', 'of', 'natural', 'language', 'data']


In [ ]:
# 문장단위 토크나이징 
from nltk.tokenize import sent_tokenize
paragraph = """ Natural language processing(nlp) is a subfield of computerscience,




"""